<a href="https://colab.research.google.com/github/monikajot/titanic/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.metrics import precision_score, accuracy_score
from sklearn.model_selection import KFold

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [ ]:
! pip install wandb
! wandb login

     |████████████████████████████████| 1.8 MB 7.7 MB/s 
     |████████████████████████████████| 181 kB 41.6 MB/s 
     |████████████████████████████████| 145 kB 58.2 MB/s 
     |████████████████████████████████| 63 kB 2.2 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=4f07c951254fc05a5418fde6bf470414561f9315d2fea6ed06e010f7dd9da81a
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb
data = pd.read_csv('/content/drive/MyDrive/kaggle-titanic/train.csv')
# survived 342, didn't 549
test_data = pd.read_csv('/content/drive/MyDrive/kaggle-titanic/test.csv', index_col='PassengerId')
# data['Fare'] is  revenue earned from transporting passengers on the public transit system,
# including a cash fare, a donation received instead of a set fare, and an advance fare received from the sale of a coupon, token, or pass.
# X_train = ('Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked')

In [ ]:
test_data[:5], len(test_data)

(             Pclass                                          Name     Sex  \
 PassengerId                                                                 
 892               3                              Kelly, Mr. James    male   
 893               3              Wilkes, Mrs. James (Ellen Needs)  female   
 894               2                     Myles, Mr. Thomas Francis    male   
 895               3                              Wirz, Mr. Albert    male   
 896               3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   
 
               Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
 PassengerId                                                       
 892          34.5      0      0   330911   7.8292   NaN        Q  
 893          47.0      1      0   363272   7.0000   NaN        S  
 894          62.0      0      0   240276   9.6875   NaN        Q  
 895          27.0      0      0   315154   8.6625   NaN        S  
 896          22.0      1      1  3101298  1

In [ ]:
test_data[test_data['Fare'].isnull()]

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,
1044,3,"Storey, Mr. Thomas",male,60.5,0,0,3701,NaN,NaN,S


The columns we keep from the data are Pclass, Sex, Age, sibSp, Parch, Fare

### baseline


In [ ]:
# if class 1 - survived, if class 2,3 didnt survive
c= []
for i in X_train['Pclass']:
    if i==1:
        c.append(1)
    else:
        c.append(0)
y_pred_b1 = pd.Series( c, dtype=int)


# if class 1,2 - survived, if class 3, didnt survive
c= []
for i in X_train['Pclass']:
    if i==1 or i==2:
        c.append(1)
    else:
        c.append(0)
y_pred_b2 = pd.Series( c, dtype=int)

NameError: ignored

In [ ]:
#get accuracy
from sklearn.metrics import mean_squared_error
a1 =mean_squared_error(y_pred_b1, y_train)

a2= mean_squared_error(y_pred_b2, y_train)
a1, a2 #baseline accuracy is 33.5%

## preprocessing

- dropped cabin
- nan age is average (?) #TODO could improve by putting in smt else?

In [ ]:
class Preprocessor():
    def __init__(self) -> None:
        self.scaler = MinMaxScaler()
        self.y_scaler = MinMaxScaler()
        # X, _ = self._preprocess(x, training=True)

    def _preprocess(self, x, y=None, training=False):
            """
            Preprocess input of the network.
            Arguments:
                - x {pd.DataFrame} -- Raw input array of shape
                    (batch_size, input_size).
                - y {pd.DataFrame} -- Raw target array of shape (batch_size, 1).
                - training {boolean} -- Boolean indicating if we are training or
                    testing the model.
            Returns:
                - {torch.tensor} -- Preprocessed input array of size
                    (batch_size, input_size).
                - {torch.tensor} -- Preprocessed target array of size
                    (batch_size, 1).
            """
            # Return preprocessed x and y, return None for y if it was None
            #turn male 1, female 0
            # Binary encode

            x.loc[x['Sex'] == 'male', 'Sex'] = 1
            x.loc[x['Sex'] == 'female', 'Sex'] = 0

            x.loc[x['Embarked'] == 'C', 'Embarked'] = 0
            x.loc[x['Embarked'] == 'Q', 'Embarked'] = 1
            x.loc[x['Embarked'] == 'S', 'Embarked'] = 2
            mu = x.loc[:, 'Embarked'].mean(skipna=True)
            x.loc[:, 'Embarked'].fillna(mu, inplace=True)

            c = x['Age'].mean()
            x['Age'].fillna(c, inplace=True)

            x['Fare'].fillna(x['Fare'].mean(), inplace=True)
            if not training:
                self.test_idx = x.index
                print(self.test_idx)

            # iqr = x['Fare'].quantile(0.75) - x['Fare'].quantile(0.25)
            # iqr2 = x['Fare'].quantile(1) -x['Fare'].quantile(0.75)
            # upper_limit = x['Fare'].quantile(0.75) +1.5 * iqr
            # lower_limit = x['Fare'].quantile(0.25) - 1.5 * iqr
            # x['Fare'] = np.where(
            #     x['Fare'] > upper_limit,
            #     upper_limit-(iqr2/x['Fare']) ,
            #     x['Fare']
            #     )


            # get min max scaling parameters from training data
            if training:
                self.scaler.fit(x)

            # apply scaling to x
            x = self.scaler.transform(x)
            if training:
                if y is not None:
                    y = y.to_numpy().reshape(-1, 1)
                    # find scale for labels in y
                    if training:

                        y=self.y_scaler.fit_transform(y)
                    else:
                        y=self.y_scaler.transform(y)

                    y_tensor = torch.from_numpy(y).float()

            x_tensor = torch.from_numpy(x).float()

            return x_tensor, (y_tensor if y is not None else None)

In [ ]:
testing_flag =False
if testing_flag:
    x = data.loc[:,('Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked')]
    y = data.loc[:,'Survived']
    Preproc = Preprocessor(x)
    X, y = Preproc._preprocess(x, y, training=True)
    X.shape, y.shape

In [ ]:

# def fit( data, split=0.25, printout=True, config=None, embarked_flag=False):
#         x = data.loc[:,('Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked')]
#         if embarked_flag:
#             x = data.loc[:,('Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare')]
#         y = data.loc[:,'Survived']
#         Preproc = Preprocessor(x)
#         X, y = Preproc._preprocess(x, y, training=True)
#         # train and val
#         X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=split, random_state=42)
#         return X_train, X_val, y_train, y_val
# X_train, X_val, y_train, y_val = fit(data)


### *linear* model

In [ ]:
# pd.set_option('display.max_rows', 500)

In [ ]:
class Simple_model(nn.Module):

    def __init__(self,  hidden=100, dropout=0.5, batch_size=64):
        super(Simple_model,self).__init__()
        self.batch_size=batch_size
        self.dropout = nn.Dropout(dropout)
        self.model = nn.Sequential(
            nn.Linear(7, hidden),
            nn.Sigmoid(),
            nn.Linear(hidden, hidden),
            nn.Sigmoid(),
            nn.Linear(hidden, hidden),
            nn.Sigmoid(),
            nn.Linear(hidden, batch_size),
            nn.Sigmoid(),
            nn.Dropout(dropout),
            nn.Linear(batch_size, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.model(x)
        return x


class Classification():

    def __init__(self, config=None):
        print(config)
        self.model = Simple_model( config.hidden, config.dropout, config.batch_size) #config.dropout,
        self.loss = nn.MSELoss()
        self.epochs = config.epochs
        self.optimizer = optim.Adam(self.model.parameters(), lr=config.lr, amsgrad=True) #TODO: read about amsgrad
        self.Prep = Preprocessor()
        self.seed = config.seed


    def init_data(self,data=data, train=True, embarked_flag=False):
        x = data.loc[:,('Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked')]
        if embarked_flag:
            x = data.loc[:,('Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare')]
        if train:
            y = data.loc[:,'Survived']
            return x, y
        return x

    # model training
    def fit(self, data,TEST_DATA=test_data, split=0.25, printout=True, config=None, embarked_flag=False):

        x, y = self.init_data()
        X, y = self.Prep._preprocess(x, y, training=True)
        X_train = X
        y_train = y
        # X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=split, random_state=42)
        # kfold = KFold(n_splits = 5, shuffle = True, random_state = 1)
        # folds = []
        # for train_index, test_index in kfold.split(X):
        # X_train, X_val = X[train_index], X[test_index]
        # y_train, y_val = y[train_index], y[test_index]
        loss_train_list = []
        loss_val_list = []
        min_loss = np.inf
        batch_indices = np.arange(len(X_train))
        np.random.seed(self.seed)
        np.random.shuffle(batch_indices)
        batch_indices = batch_indices[:-(len(X_train) % self.model.batch_size)].reshape(-1, self.model.batch_size)

        for epoch in range(config.epochs):
            running_loss = 0.0

            self.model.train()
            for ib, batch in enumerate(batch_indices):

                # sample batch from X,Y to get train_x,train_y
                # if someone could make this into a proper batch processing method, that'll be great
                X_batch = X_train[batch].clone().detach().float()
                y_batch = y_train[batch].clone().detach().float()

                self.optimizer.zero_grad()  # Zero Grad
                y_pred = self.model(X_batch)  # Forward through the model
                loss = self.loss(y_pred, y_batch)  # Calculate loss
                wandb.log({'batch_loss': loss})
                loss.backward()  # Backprop
                self.optimizer.step()  # step
                if printout:
                    running_loss += loss.item()
            if printout:
                train_loss = running_loss / batch_indices.shape[0]
                loss_train_list.append(train_loss)
                train_accuracy = self.get_accuracy(y_pred, y_batch).item()
                wandb.log({'epoch': epoch, 'accuracy': train_accuracy, 'loss': train_loss})

            # self.model.eval()
            # y_pred_val = self.model(X_val)
            # val_loss = self.loss(y_pred_val, y_val)
            # val_accuracy = self.get_accuracy(y_pred_val, y_val).item()
            # wandb.log({'epoch': epoch, 'val accuracy': val_accuracy, 'val loss': val_loss})
            # folds.append(val_accuracy)
            # loss_val_list.append(val_loss.item())

        #plot graphs
        # plt.show()
        # plt.title('Training loss')
        # plt.plot(np.arange(len(loss_train_list)), loss_train_list, label='train loss')
        # plt.legend()
        # plt.show()
        # plt.title('Validation loss')
        # plt.plot(np.arange(len(loss_val_list)), loss_val_list, label='val loss')
        # plt.legend()
        # plt.show()
        # print('Final validation accuracy: ',self.get_accuracy(y_pred_val, y_val, X_val,confusion_flag=True).item())
        # wandb.log({'fold': sum(folds)/len(folds)})
        self.predict(TEST_DATA)
        print('model predicted')

        self.save_model()
        print('model saved')


    def get_accuracy(self,y_pred, y_batch, X_val=None, confusion_flag=False):
        with torch.no_grad():
            Y = y_pred.detach().numpy().copy()
            Y[y_pred<0.5]=0
            Y[y_pred>=0.5]=1
            acc = accuracy_score(y_batch, Y)
            if confusion_flag:
                confm = confusion_matrix(y_batch.numpy(),Y)
                df_cm = pd.DataFrame(confm)
                ax = sns.heatmap(df_cm, cmap='Oranges', annot=True)
            return acc

    def eval_false(self,dfs, y_batch, Y,X_pred):
        """
        Returns a df of predicted and true values with their model inputs
        """
        dfs = pd.DataFrame(data = {'true':y_batch.squeeze(1).numpy(), 'pred': Y.squeeze(1)}, index = np.arange(len(y_batch)))
        print(dfs)
        i = 0
        columns= ['idx', 'true', 'pred', 'Pclass',"Sex", 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
        info = []
        for idx in range(len(dfs)):
            if dfs.iloc[idx,0] != dfs.iloc[idx,1]:
                x = (np.round( X_pred[idx,:]*100)/100).tolist()
                info.append([idx, dfs.loc[idx,:].values[0],dfs.loc[idx,:].values[1]]+x)
                i+=1
        info_df = pd.DataFrame(info, columns=columns)
        return info_df

    def predict(self, x):
        """
        Returns a prediction given input data
        """
        x = self.init_data(test_data, train=False)
        X, _ = self.Prep._preprocess(x, training=False)
        print(X.shape, "X \n", X)
        pred = self.model(X.float()).detach().numpy()
        print('predicted succs')
        survived = self.Prep.y_scaler.inverse_transform(pred)
        Y = survived.copy()
        Y[survived<0.5]=0
        Y[survived>=0.5]=1
        # print(survived, type(survived))
        print(type(survived), survived.shape, type(self.Prep.test_idx), self.Prep.test_idx.shape )
        df = pd.DataFrame({ 'PassengerId':self.Prep.test_idx, 'Survived': Y.squeeze(1).astype(int)})
        print(df)
        df.to_csv('submission.csv')

    def save_model(self):
        torch.save(self.model.state_dict(), 'Titanic_model')


    def load_model(self,f):
        model = Simple_model(hidden=256)
        model.eval()
        model.load_state_dict(torch.load(f))
        return model



In [ ]:
sweep_config = {
    'method': 'random'
    }
metric = {
    'name': 'loss',
    'goal': 'minimize'
    }

sweep_config['metric'] = metric

parameters_dict = {
    'hidden': {
        'values': [64]
        },
    'epochs': {
        'value': 100},
    'lr': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0.002,
        'max': 0.0025
        },
    'dropout': {
          'values': [0.2, 0.27,  0.3]
        },
    'batch_size': {
        # integers between 32 and 256
        # with evenly-distributed logarithms
        # Quantized log uniform. Returns round(X / q) * q where X is log_uniform_values. q defaults to 1.
        'distribution': 'q_log_uniform_values',
        'q': 8,
        'min': 32,
        'max': 40,
      },
      'seed': {
        'value' : 3# [1,2,3,4,15,23,65,11,111,123,987]
      }
    }

sweep_config['parameters'] = parameters_dict
sweep_id = wandb.sweep(sweep_config, project="titanic-sweeps-3")
# wandb.init(project="titanic", entity="monikajot")


Create sweep with ID: 06x6gk4g
Sweep URL: https://wandb.ai/monikajot/titanic-sweeps-3/sweeps/06x6gk4g


In [ ]:
def train(config=None):
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        classify = Classification(config)
        classify.fit(data,config=config)
        print('##################################################### \n #####################################################')

In [ ]:
import os
os.environ['WANDB_MODE'] = 'offline'
wandb.agent(sweep_id, train, count=1)
# .87 {'batch_size': 32, 'dropout': 0.35, 'epochs': 70, 'hidden': 64, 'lr': 0.002432054780211515}
# .87 {'batch_size': 40, 'dropout': 0.27, 'epochs': 70, 'hidden': 64, 'lr': 0.000432322433300812}
# .9 {'batch_size': 32, 'dropout': 0.2, 'epochs': 100, 'hidden': 64, 'lr': 0.0022233305920332453, 'seed': 3}

wandb: Agent Starting Run: xegd8bnz with config:
wandb: 	batch_size: 40
wandb: 	dropout: 0.27
wandb: 	epochs: 100
wandb: 	hidden: 64
wandb: 	lr: 0.0023609049711815766
wandb: 	seed: 3


{'batch_size': 40, 'dropout': 0.27, 'epochs': 100, 'hidden': 64, 'lr': 0.0023609049711815766, 'seed': 3}
Int64Index([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,
            ...
            1300, 1301, 1302, 1303, 1304, 1305, 1306, 1307, 1308, 1309],
           dtype='int64', name='PassengerId', length=418)
torch.Size([418, 7]) X 
 tensor([[1.0000, 1.0000, 0.4282,  ..., 0.0000, 0.0153, 0.5000],
        [1.0000, 0.0000, 0.5853,  ..., 0.0000, 0.0137, 1.0000],
        [0.5000, 1.0000, 0.7738,  ..., 0.0000, 0.0189, 0.5000],
        ...,
        [1.0000, 1.0000, 0.4785,  ..., 0.0000, 0.0142, 1.0000],
        [1.0000, 1.0000, 0.3751,  ..., 0.0000, 0.0157, 1.0000],
        [1.0000, 1.0000, 0.3751,  ..., 0.1667, 0.0436, 0.0000]])
predicted succs
<class 'numpy.ndarray'> (418, 1) <class 'pandas.core.indexes.numeric.Int64Index'> (418,)
     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            

accuracy,▁▁▄█▇███▇█▇█▇█▇▇▇▇▇█▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
batch_loss,▇█▆▅▆▄▁▂▄▃▆▄▄▃▃▅▄▄▄▁▂▅▃▆▄▄▃▃▃▄▄▅▁▃▅▄▆▄▁▃
epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,██▆▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy,0.9
batch_loss,0.07699
epoch,99
loss,0.13742


# --- THE END ---

In [ ]:
x = pd.read_csv('submission.csv')
x

,Unnamed: 0,PassengerId,Survived
0,0,892,0
1,1,893,0
2,2,894,0
3,3,895,0
4,4,896,0
...,...,...,...
413,413,1305,0
414,414,1306,1
415,415,1307,0
416,416,1308,0


In [ ]:
with open('X_val.obj', 'rb') as f:
    X_val = pickle.load(f)

with open('y_val.obj', 'rb') as f:
    y_val = pickle.load(f)

model = Simple_model(hidden=256)
model.eval()

model.load_state_dict(torch.load("/content/Titanic_model"))
y_pred_val = model(X_val)
val_loss = torch.nn.MSELoss()
loss = val_loss(y_pred_val, y_val)
C = Classification()
val_accuracy = C.get_accuracy(y_pred_val, y_val).item()
loss, val_accuracy

(tensor(0.1398, grad_fn=<MseLossBackward0>), 0.8161434977578476)

In [ ]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits = 3, shuffle = True, random_state = 1)

# for train, test in kfold.split(X):
#     print(train, test)
#     print()
	# X_train, X_test = X[train_index], X[test_index]
  	# y_train, y_test = y[train_index], y[test_index]



In [ ]:
columns= ['idx', 'true', 'pred', 'Pclass',"Sex", 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
info = []
for idx in range(len(dfs)):
    if dfs.iloc[idx,0] != dfs.iloc[idx,1]:
        x = (np.round( X_pred[idx,:]*100)/100).tolist()
        info.append([idx, dfs.loc[idx,:].values[0],dfs.loc[idx,:].values[1]]+x)
        i+=1
info_df = pd.DataFrame(info, columns=columns)
